In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import *
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, ZeroPadding2D
from keras.layers import BatchNormalization
from keras import initializers
from keras import regularizers
from keras.layers.advanced_activations import LeakyReLU

import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam


Using TensorFlow backend.


In [2]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

Instructions for updating:
Colocations handled automatically by placer.


C:\Programy\Anaconda3\lib\site-packages\keras_applications\mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [3]:
# Sizes of the scaled images
height = 512
width = 512

ROOT_PATH = '../../ISIC-Archive-Downloader/Data/'


train_directory_path = ROOT_PATH + 'train2'

test_directory_path = ROOT_PATH + 'test2'

In [4]:
train_data = ImageDataGenerator(
     rescale=1 / 255
)#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True)

#train_data = ImageDataGenerator()
batch_size = 16
test_datagen = ImageDataGenerator()

train_generator = train_data.flow_from_directory(
    train_directory_path,
    target_size=(width, height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    test_directory_path,
    target_size=(width, height),
    batch_size=batch_size,
    class_mode='categorical')

Found 3962 images belonging to 2 classes.
Found 1752 images belonging to 2 classes.


In [5]:
model=Model(inputs=base_model.input,outputs=preds)


for layer in model.layers[:-8]:
    layer.trainable=False

In [6]:

#model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [7]:
step_size_train=train_generator.n//train_generator.batch_size
step_size_validate = validation_generator.n // validation_generator.batch_size
#model.fit_generator(generator=train_generator,
#                   steps_per_epoch=step_size_train,
#                   epochs=2)

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

In [10]:

from keras.models import load_model
model = load_model('my_model.h5')
model.evaluate_generator(validation_generator, 3, verbose=1)

AbortedError: Operation received an exception:Status: 3, message: could not create a dilated convolution forward descriptor, in file tensorflow/core/kernels/mkl_conv_ops.cc:1111
	 [[{{node conv1_2/convolution}}]]